<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_MLP_PGD_SHAP_detection_BIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 4.3 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [7]:
pgd_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/24_feature/pgd/pgd_mlp_bin_feature'
#pgd_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/pgd/adv_pgd_target.csv'

In [8]:
pgd_df = pd.read_csv(pgd_path)

In [9]:
pgd_df.columns = feature_list
pgd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

# Load DL-based IDS model

In [10]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100, random_state = 42))

In [12]:
pgd_1000 = pgd_df.head(1000)

In [13]:
shap_values = explainer.shap_values(pgd_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
print(shap_values[0])

[[-0.01473223  0.01174439  0.         ...  0.08980513  0.
   0.01846874]
 [-0.00678737  0.          0.         ...  0.09171713  0.
   0.        ]
 [-0.01042646  0.          0.         ...  0.1000485   0.
  -0.02582398]
 ...
 [-0.01279952  0.02145395 -0.00558308 ...  0.07448076 -0.04035739
   0.00833362]
 [ 0.         -0.004425    0.         ...  0.07203009  0.00912544
   0.        ]
 [-0.01238782  0.          0.         ...  0.          0.0357647
   0.08447582]]


In [15]:
type(shap_values[0])

numpy.ndarray

In [16]:
data_df = pd.DataFrame(shap_values[0])

In [17]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,-0.014732,0.011744,0.000000,0.000000,-0.009278,0.000000,0.000000,0.005575,0.009464,0.000000,...,0.000000,0.0,0.079029,-0.049693,0.031336,0.174497,0.000000,0.089805,0.000000,0.018469
1,-0.006787,0.000000,0.000000,-0.021434,0.000000,-0.005636,0.009382,0.000000,0.000000,0.000000,...,-0.012221,0.0,0.008483,-0.039968,0.036355,0.031094,0.000000,0.091717,0.000000,0.000000
2,-0.010426,0.000000,0.000000,0.000000,0.000000,0.011686,-0.001920,0.010051,0.000000,0.004833,...,-0.006032,0.0,0.007647,-0.047337,0.000000,0.050765,-0.011647,0.100048,0.000000,-0.025824
3,-0.008982,-0.004130,-0.007585,0.010104,0.000000,0.000000,0.002620,-0.004705,-0.003416,0.007709,...,0.000000,0.0,0.015528,-0.025381,0.032851,0.043814,0.000000,0.055047,-0.010202,-0.009728
4,0.011083,-0.003549,-0.006350,-0.006397,0.000000,0.000000,0.000000,-0.013491,-0.006847,-0.007967,...,-0.006431,0.0,0.038468,-0.054022,0.068598,0.116031,-0.046180,0.048447,0.012261,-0.086004


In [ ]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_pgd/1000_samples_[0].csv', index=False)

In [18]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,-0.014732,0.011744,0.000000,0.000000,-0.009278,0.000000,0.000000,0.005575,0.009464,0.000000,...,0.000000,0.0,0.079029,-0.049693,0.031336,0.174497,0.000000,0.089805,0.000000,0.018469
1,-0.006787,0.000000,0.000000,-0.021434,0.000000,-0.005636,0.009382,0.000000,0.000000,0.000000,...,-0.012221,0.0,0.008483,-0.039968,0.036355,0.031094,0.000000,0.091717,0.000000,0.000000
2,-0.010426,0.000000,0.000000,0.000000,0.000000,0.011686,-0.001920,0.010051,0.000000,0.004833,...,-0.006032,0.0,0.007647,-0.047337,0.000000,0.050765,-0.011647,0.100048,0.000000,-0.025824
3,-0.008982,-0.004130,-0.007585,0.010104,0.000000,0.000000,0.002620,-0.004705,-0.003416,0.007709,...,0.000000,0.0,0.015528,-0.025381,0.032851,0.043814,0.000000,0.055047,-0.010202,-0.009728
4,0.011083,-0.003549,-0.006350,-0.006397,0.000000,0.000000,0.000000,-0.013491,-0.006847,-0.007967,...,-0.006431,0.0,0.038468,-0.054022,0.068598,0.116031,-0.046180,0.048447,0.012261,-0.086004


In [19]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [ ]:
print(data_dict)

In [20]:
len(shap_values[0])

1000

In [21]:
len(data_dict)

1000

# Detection phase

In [22]:
import os
import pandas as pd

In [26]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Down/Up Ratio,874
1,Bwd Pkt Len Mean,841
2,Init Bwd Win Byts,780
3,Bwd Seg Size Avg,764
4,Fwd Pkt Len Min,746
5,Bwd Pkt Len Max,581
6,Idle Min,508
7,Bwd Pkt Len Std,455
8,ACK Flag Cnt,405
9,Flow Duration,377


## Detection rate (top 50) = 986/1000

In [40]:
whitelist = whitelist_df.iloc[:, 0][:50].tolist()
print(whitelist)

['Down/Up Ratio', 'Bwd Pkt Len Mean', 'Init Bwd Win Byts', 'Bwd Seg Size Avg', 'Fwd Pkt Len Min', 'Bwd Pkt Len Max', 'Idle Min', 'Bwd Pkt Len Std', 'ACK Flag Cnt', 'Flow Duration', 'Flow IAT Std', 'Idle Mean', 'SYN Flag Cnt', 'Bwd IAT Std', 'Tot Bwd Pkts', 'Tot Fwd Pkts', 'PSH Flag Cnt', 'Bwd IAT Max', 'TotLen Bwd Pkts', 'Fwd IAT Tot', 'TotLen Fwd Pkts', 'Bwd IAT Tot', 'Bwd Pkts/s', 'Idle Max', 'Flow IAT Mean', 'Bwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Bwd IAT Min', 'Flow IAT Max', 'Fwd Pkt Len Std', 'Bwd Pkt Len Min', 'Active Min', 'Fwd IAT Std', 'Subflow Bwd Pkts', 'Active Max', 'Pkt Len Max', 'Active Std', 'Active Mean', 'Pkt Len Std', 'Flow IAT Min', 'Pkt Len Min', 'Fwd IAT Max', 'Pkt Len Mean', 'Fwd IAT Min', 'Subflow Bwd Byts', 'Pkt Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Pkt Len Var']


In [41]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:986


## Detection rate (top 60) = 803/1000

In [38]:
whitelist2 = whitelist_df.iloc[:, 0][:60].tolist()
print(whitelist2)

['Down/Up Ratio', 'Bwd Pkt Len Mean', 'Init Bwd Win Byts', 'Bwd Seg Size Avg', 'Fwd Pkt Len Min', 'Bwd Pkt Len Max', 'Idle Min', 'Bwd Pkt Len Std', 'ACK Flag Cnt', 'Flow Duration', 'Flow IAT Std', 'Idle Mean', 'SYN Flag Cnt', 'Bwd IAT Std', 'Tot Bwd Pkts', 'Tot Fwd Pkts', 'PSH Flag Cnt', 'Bwd IAT Max', 'TotLen Bwd Pkts', 'Fwd IAT Tot', 'TotLen Fwd Pkts', 'Bwd IAT Tot', 'Bwd Pkts/s', 'Idle Max', 'Flow IAT Mean', 'Bwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Bwd IAT Min', 'Flow IAT Max', 'Fwd Pkt Len Std', 'Bwd Pkt Len Min', 'Active Min', 'Fwd IAT Std', 'Subflow Bwd Pkts', 'Active Max', 'Pkt Len Max', 'Active Std', 'Active Mean', 'Pkt Len Std', 'Flow IAT Min', 'Pkt Len Min', 'Fwd IAT Max', 'Pkt Len Mean', 'Fwd IAT Min', 'Subflow Bwd Byts', 'Pkt Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Pkt Len Var', 'Bwd IAT Mean', 'Fwd Pkts/s', 'Fwd IAT Mean', 'Flow Byts/s', 'Fwd Header Len', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'FIN Flag Cnt', 'Fwd Seg Size Avg', 'URG Flag Cnt']


In [39]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:803
